# Initialize

In [ ]:
!pip install p3_data openpyxl

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pylab as plt
import matplotlib.dates as mdates
import matplotlib.cm as cm
import seaborn as sns
import json
from io import StringIO
import importlib
import re

In [ ]:
import p3_data
from p3_data import (glob_file_list , load_json_from_file, merge_dicts, plot_groups, 
                    get_varying_column_names, filter_dataframe, take_varying_columns,
                    load_json_records_as_dataframe, flatten_multiindex_columns,
                    regex_first_group)

In [ ]:
pd.set_option('display.max_rows', 200)

# Load and Clean Results

In [ ]:
# Load result files from P3 Test Driver
src_files = []
# src_files += ['../../mnt/isilon/data/genomics/summary/*.json.bz2']
src_files += ['../data/parabricks/summary/*.json.bz2']
src_files += ['../data/parabricks/8440*/summary/*.json.bz2']
raw_df = load_json_records_as_dataframe(src=src_files, ignore_error=True)

In [ ]:
local_dirs = ['/raid','/tmp']
isilon_dirs = ['/mnt/isilon']
hostname_to_gpu_map = {
    'isilon': 10,
    'dgx2-1': 16,
    'dgx2-2': 16,
    'dgx2-3': 16,
}

In [ ]:
# Clean raw results
def clean_result(result):
    r = result.copy()
    r['clean'] = False
    try:
        r['utc_begin'] = pd.to_datetime(r['utc_begin'], utc=True)
        r['utc_end'] = pd.to_datetime(r['utc_end'], utc=True)
        r['total_minutes'] = r['elapsed_sec'] / 60.0        
        r['num_gpus_per_server'] = hostname_to_gpu_map.get(r['hostname'], np.nan)        
        r['num_servers'] = r['num_gpus'] / r['num_gpus_per_server']  # fraction of server used for this job
        r['total_server_minutes'] = r['num_servers'] * r['elapsed_sec'] / 60.0
        r['total_gpu_minutes'] = r['num_gpus'] * r['elapsed_sec'] / 60.0

        data_layout = np.nan
        r['is_input_isilon'] = any([d in r.args['input_dir'] for d in isilon_dirs])
        r['is_input_local'] = any([d in r.args['input_dir'] for d in local_dirs])
        r['is_temp_isilon'] = any([d in r.args['temp_dir'] for d in isilon_dirs])
        r['is_temp_local'] = any([d in r.args['temp_dir'] for d in local_dirs])
        if r.is_input_isilon and r.is_temp_local:
            data_layout = 'mixed'
        elif r.is_input_isilon and r.is_temp_isilon:
            data_layout = 'all_isilon'
        elif r.is_input_local and r.is_temp_local:
            data_layout = 'all_local'        
        r['data_layout'] = data_layout
        
        if not ('error' in r and r['error']==True):
            r['error'] = False
        
        if 'fq2bam_result' in r and isinstance(r.fq2bam_result, dict) and not r.fq2bam_result['error']:
            r['parabricks_version'] = regex_first_group('Version v([^ ]+)', r['fq2bam_result']['errors'], search=True)
            r['bwa_mem_sec'] = float(regex_first_group('GPU-BWA Mem time: (.*) seconds', r.fq2bam_result['errors'], search=True))
            r['fq2bam_sec'] = r.fq2bam_result['elapsed_sec']
            if r.fq2bam_result['error']: r['error'] = True
            # GPU-BWA Mem + Sorting + MarkingDups + BQSR Generation + BAM writing
            r['bwa_mem_to_bam_writing_sec'] = float(regex_first_group('Processing time: (.*) seconds', r.fq2bam_result['errors'], search=True))
            # Sorting + MarkingDups + BQSR Generation + BAM writing
            r['sorting_to_bam_writing_sec'] = r['bwa_mem_to_bam_writing_sec'] - r['bwa_mem_sec']
            r['bwa_mem_minutes'] = r['bwa_mem_sec'] / 60.0
            r['bwa_mem_to_bam_writing_minutes'] = r['bwa_mem_to_bam_writing_sec'] / 60.0
            r['sorting_to_bam_writing_minutes'] = r['sorting_to_bam_writing_sec'] / 60.0
            r['bwa_mem_to_bam_writing_server_minutes'] = r['num_servers'] * r['bwa_mem_to_bam_writing_sec'] / 60.0
            r['bwa_mem_to_bam_writing_gpu_minutes'] = r['num_gpus'] * r['bwa_mem_to_bam_writing_sec'] / 60.0
            r['fq_file_size_bytes'] = np.sum(r.fq_file_sizes)
            r['fq_file_size_GB'] = r['fq_file_size_bytes'] * 1e-9
            r['bam_file_size_GB'] = r['bam_file_size_bytes'] * 1e-9
            r['bwa_mem_to_bam_writing_utc_begin'] = pd.to_datetime(r.fq2bam_result['utc_begin'], utc=True)
            r['bwa_mem_to_bam_writing_utc_end'] = pd.to_datetime(r.fq2bam_result['utc_end'], utc=True)

        if 'germline_result' in r and isinstance(r.germline_result, dict) and not r.germline_result['error']:
            r['parabricks_version'] = regex_first_group('Version v([^ ]+)', r['germline_result']['errors'], search=True)
            r['bwa_mem_sec'] = float(regex_first_group('GPU-BWA Mem time: (.*) seconds', r.germline_result['errors'], search=True))
            r['germline_sec'] = r.germline_result['elapsed_sec']
            if r.germline_result['error']: r['error'] = True
            # GPU-BWA Mem + Sorting + MarkingDups + BQSR Generation + BAM writing
            r['bwa_mem_to_bam_writing_sec'] = float(regex_first_group('Processing time: (.*) seconds', r.germline_result['errors'], search=True))
            # Sorting + MarkingDups + BQSR Generation + BAM writing
            r['sorting_to_bam_writing_sec'] = r['bwa_mem_to_bam_writing_sec'] - r['bwa_mem_sec']
            r['haplotypecaller_sec'] = float(regex_first_group('Total time taken: (.*)', r.germline_result['errors'], search=True))
            r['germline_minutes'] = r['germline_sec'] / 60.0
            r['bwa_mem_minutes'] = r['bwa_mem_sec'] / 60.0
            r['bwa_mem_to_bam_writing_minutes'] = r['bwa_mem_to_bam_writing_sec'] / 60.0
            r['sorting_to_bam_writing_minutes'] = r['sorting_to_bam_writing_sec'] / 60.0
            r['haplotypecaller_minutes'] = r['haplotypecaller_sec'] / 60.0
            r['bwa_mem_to_bam_writing_server_minutes'] = r['num_servers'] * r['bwa_mem_to_bam_writing_sec'] / 60.0
            r['bwa_mem_to_bam_writing_gpu_minutes'] = r['num_gpus'] * r['bwa_mem_to_bam_writing_sec'] / 60.0
            r['haplotypecaller_server_minutes'] = r['num_servers'] * r['haplotypecaller_sec'] / 60.0
            r['haplotypecaller_gpu_minutes'] = r['num_gpus'] * r['haplotypecaller_sec'] / 60.0
            r['fq_file_size_bytes'] = np.sum(r.fq_file_sizes)
            r['fq_file_size_GB'] = r['fq_file_size_bytes'] * 1e-9
            r['bam_file_size_GB'] = r['bam_file_size_bytes'] * 1e-9
            r['haplotypecaller_gvcf_file_size_GB'] = r['haplotypecaller_gvcf_file_size_bytes'] * 1e-9
            r['bwa_mem_to_bam_writing_utc_begin'] = pd.to_datetime(r.germline_result['utc_begin'], utc=True)
            r['bwa_mem_to_bam_writing_utc_end'] = r['bwa_mem_to_bam_writing_utc_begin'] + pd.to_timedelta(r['bwa_mem_to_bam_writing_sec'], unit='sec')
            r['bwa_mem_utc_end'] = r['bwa_mem_to_bam_writing_utc_begin'] + pd.to_timedelta(r['bwa_mem_sec'], unit='sec')
            r['haplotypecaller_utc_begin'] = r['bwa_mem_to_bam_writing_utc_end']
            r['haplotypecaller_utc_end'] = pd.to_datetime(r.germline_result['utc_end'], utc=True)
                
        if 'deepvariant_result' in r and isinstance(r.deepvariant_result, dict) and not r.deepvariant_result['error']:
            r['deepvariant_sec'] = r.deepvariant_result['elapsed_sec']
            if r.deepvariant_result['error']: r['error'] = True
            r['deepvariant_minutes'] = r['deepvariant_sec'] / 60.0
            r['deepvariant_server_minutes'] = r['num_servers'] * r['deepvariant_sec'] / 60.0            
            r['deepvariant_gpu_minutes'] = r['num_gpus'] * r['deepvariant_sec'] / 60.0            
            r['deepvariant_gvcf_file_size_GB'] = r['deepvariant_gvcf_file_size_bytes'] * 1e-9
            r['deepvariant_utc_begin'] = pd.to_datetime(r.deepvariant_result['utc_begin'], utc=True)
            r['deepvariant_utc_end'] = pd.to_datetime(r.deepvariant_result['utc_end'], utc=True)
       
        r['clean'] = True
    except Exception as e:
        print('ERROR: %s: %s' % (r['loaded_filename'], e))
        raise e
    return pd.Series(r)

In [ ]:
r = clean_result(raw_df.iloc[-1])
pd.DataFrame(r)

In [ ]:
clean1_df = raw_df.apply(clean_result, axis=1)
clean1_df = clean1_df.set_index('record_uuid', drop=False)
clean1_df = clean1_df.sort_values(['utc_begin'])

In [ ]:
clean1_df = clean1_df[clean1_df.clean==True]
clean1_df = clean1_df[clean1_df.error==False]

In [ ]:
len(clean1_df)

In [ ]:
sample_ids_df = pd.read_csv('sample_ids_300.csv').set_index(['sample_id'])
sample_ids_df.head()

In [ ]:
clean_df = clean1_df.join(sample_ids_df, on=['sample_id'])
clean_df['coverage_binned'] = (pd.cut(clean_df['coverage'], bins=[35,40,45,50,51.7,68,80,85], right=False)
                               .apply(lambda x: np.round(x.left)).astype(float))
clean_df.head(3).T

# Explore Data

In [ ]:
clean_df.parabricks_version.value_counts()

In [ ]:
filt_df = filter_dataframe(
    clean_df,
    num_gpus=5,
#     coverage_binned=50,
    total_minutes=(15,1000),
#     parabricks_version='2.3.2',
)
len(filt_df)

In [ ]:
filt_df.iloc[0].T

In [ ]:
# Define columns that identify test parameters
param_cols = [
    'sample_id',
    'coverage',
    'num_gpus',
    'parabricks_version',
]

In [ ]:
# Define columns that are the output of the experiments
output_cols = [
    'utc_begin',
    'bwa_mem_to_bam_writing_minutes',
    'haplotypecaller_minutes',
    'deepvariant_minutes',
]

In [ ]:
cols = param_cols + output_cols

In [ ]:
# View most recent results
filt_df[cols].tail(10).T

In [ ]:
clean_df[cols].set_index(['sample_id']).sort_values(['coverage'])

# Compare different GPUs/sample

In [ ]:
filt_df = filter_dataframe(
    clean_df,
    batch_uuid=[
        'c9bd8b49-42ec-48de-a80f-99bcd0770964', # DGX-2, 8 GPUs
        '6f9eeed6-3b09-4000-9d37-6eed11281f83', # DGX-2, 4 GPUs
        '84a1ead5-f050-4e9a-ae31-9034fa9bf558', # 8440, 5 GPUs
    ],
)
len(filt_df)

In [ ]:
filt_df[cols].set_index(['coverage','sample_id','num_gpus']).unstack(['num_gpus'])

In [ ]:
groupby_cols = [
    'coverage_binned',
    'num_gpus',
]
df = filt_df.groupby(groupby_cols).agg({'coverage': ['min','mean','max','size']}).unstack(['num_gpus'])
df.to_excel('results_coverage.xlsx')
df

In [ ]:
groupby_cols = [
    'coverage_binned',
    'num_gpus',
]
agg_cols = [
    'total_minutes',
    'bwa_mem_to_bam_writing_minutes',
    'haplotypecaller_minutes',
    'deepvariant_minutes',
]
df = filt_df[groupby_cols + agg_cols].groupby(groupby_cols).agg(['min','mean','max']).unstack(['num_gpus'])
df.to_excel('results_wall_clock_times.xlsx')
df.T

In [ ]:
ax = df[('bwa_mem_to_bam_writing_minutes','mean')].plot(
    style='-x', 
    ylim=[0,None], 
    title='BWA, Wall Clock Time, Mixed Data Layout',
    grid=True,
)
ax.figure.savefig('bwa_wall_clock_times.svg')

In [ ]:
ax = df[('haplotypecaller_minutes','mean')].plot(
    style='-x', 
    ylim=[0,None], 
    title='HC, Wall Clock Time, Mixed Data Layout',
    grid=True,
)
ax.figure.savefig('hc_wall_clock_times.svg')

In [ ]:
ax = df[('deepvariant_minutes','mean')].plot(
    style='-x', 
    ylim=[0,None], 
    title='DV, Wall Clock Time, Mixed Data Layout',
    grid=True,
)
ax.figure.savefig('dv_wall_clock_times.svg')

## Server-Minutes

In [ ]:
groupby_cols = [
    'coverage_binned',
    'num_gpus',
]
agg_cols = [
    'total_server_minutes',
    'bwa_mem_to_bam_writing_server_minutes',
    'haplotypecaller_server_minutes',
    'deepvariant_server_minutes',
]
df = filt_df[groupby_cols + agg_cols].groupby(groupby_cols).agg(['min','mean','max']).unstack(['num_gpus'])
df.to_excel('results_server_minutes.xlsx')
df.T

In [ ]:
ax = df[('bwa_mem_to_bam_writing_server_minutes','mean')].plot(
    style='-x', 
    ylim=[0,None], 
    title='BWA, Server-Minutes, Mixed Data Layout',
    grid=True,
)
ax.figure.savefig('bwa_server_minutes.svg')

In [ ]:
ax = df[('haplotypecaller_server_minutes','mean')].plot(
    style='-x', 
    ylim=[0,None], 
    title='HC, Server-Minutes, Mixed Data Layout',
    grid=True,
)
ax.figure.savefig('hc_server_minutes.svg')

In [ ]:
ax = df[('deepvariant_server_minutes','mean')].plot(
    style='-x', 
    ylim=[0,None], 
    title='DV, Server-Minutes, Mixed Data Layout',
    grid=True,
)
ax.figure.savefig('dv_server_minutes.svg')

In [ ]:
ax = df[('total_server_minutes','mean')].plot(
    style='-x', 
    ylim=[0,None], 
    title='BWA+HC+DV Total Server-Minutes, Mixed Data Layout',
    grid=True,
)
ax.figure.savefig('total_server_minutes.svg')

## GPU-Minutes

In [ ]:
groupby_cols = [
    'coverage_binned',
    'num_gpus',
]
agg_cols = [
    'total_gpu_minutes',
    'bwa_mem_to_bam_writing_gpu_minutes',
    'haplotypecaller_gpu_minutes',
    'deepvariant_gpu_minutes',
]
df = filt_df[groupby_cols + agg_cols].groupby(groupby_cols).agg(['min','mean','max']).unstack(['num_gpus'])
df.to_excel('results_gpu_minutes.xlsx')
df.T

In [ ]:
ax = df[('bwa_mem_to_bam_writing_gpu_minutes','mean')].plot(
    style='-x', 
    ylim=[0,None], 
    title='BWA, GPU-Minutes, Mixed Data Layout',
    grid=True,
)
ax.figure.savefig('bwa_gpu_minutes.svg')

In [ ]:
ax = df[('haplotypecaller_gpu_minutes','mean')].plot(
    style='-x', 
    ylim=[0,None], 
    title='HC, GPU-Minutes, Mixed Data Layout',
    grid=True,
)
ax.figure.savefig('hc_gpu_minutes.svg')

In [ ]:
ax = df[('deepvariant_gpu_minutes','mean')].plot(
    style='-x', 
    ylim=[0,None], 
    title='DV, GPU-Minutes, Mixed Data Layout',
    grid=True,
)
ax.figure.savefig('dv_gpu_minutes.svg')

In [ ]:
ax = df[('total_gpu_minutes','mean')].plot(
    style='-x', 
    ylim=[0,None], 
    title='BWA+HC+DV Total GPU-Minutes, Mixed Data Layout',
    grid=True,
)
ax.figure.savefig('total_gpu_minutes.svg')

## File Sizes

In [ ]:
groupby_cols = [
    'coverage_binned',
]
agg_cols = [
    'fq_file_size_GB',
    'bam_file_size_GB',
    'haplotypecaller_gvcf_file_size_GB',
    'deepvariant_gvcf_file_size_GB',
]
df = filt_df[groupby_cols + agg_cols].groupby(groupby_cols).agg(['min','mean','max'])
df.to_excel('results_file_sizes.xlsx')
df

In [ ]:
ax = df[[('fq_file_size_GB','mean'),
    ('bam_file_size_GB','mean'),
    ('haplotypecaller_gvcf_file_size_GB','mean'),
    ('deepvariant_gvcf_file_size_GB','mean')]].plot(
    style='-x', 
    figsize=(12,4),
    ylim=[0,None], 
    title='File Sizes',
    grid=True,
)
ax.figure.savefig('file_sizes.svg')

In [ ]:
filt_df[['sample_id', 'coverage', 'num_gpus']].sort_values(['coverage'])

In [ ]:
filt3_df = filter_dataframe(
    filt_df,
#     coverage=(68,85),
    hostname='dgx2-1',
    num_gpus=16,
)
len(filt3_df)

In [ ]:
# Get timestamps of different phases.
filt3_df[[
    'sample_id',
    'coverage',
    'hostname',
    'bwa_mem_to_bam_writing_utc_begin',
    'bwa_mem_utc_end',
    'haplotypecaller_utc_begin',
    'deepvariant_utc_begin',
    'deepvariant_utc_end',
]].sort_values(['bwa_mem_to_bam_writing_utc_begin'])

In [ ]:
filt4_df = filter_dataframe(
    filt_df,
    coverage_binned=40,
#     num_gpus=4,
    sample_id='LP6005443-DNA_H08',
)
len(filt4_df)

In [ ]:
filt4_df[['sample_id','batch_uuid','coverage','num_gpus']]

# Compare Data Layouts

In [ ]:
# Define columns that identify test parameters
param_cols = [
    'sample_id',
    'coverage',
    'data_layout',
]

In [ ]:
# Define columns that are the output of the experiments
output_cols = [
    'bwa_mem_to_bam_writing_minutes',
]

In [ ]:
cols = param_cols + output_cols

In [ ]:
filt_df = filter_dataframe(
    clean_df,
    batch_uuid=[
        '791fbb1a-ddc9-4fff-aedf-b638c94a116f', # all local
        '69510f6c-8201-47e3-8859-dca272998f36', # all Isilon
        '155ddaab-addd-46a4-b608-319c071d61d2', # mixed
    ],
)
len(filt_df)

In [ ]:
filt_df[cols].set_index(['coverage','sample_id','data_layout']).unstack(['data_layout'])

In [ ]:
groupby_cols = [
    'coverage_binned',
    'data_layout',
]
agg_cols = [
    'bwa_mem_to_bam_writing_minutes',
]
df = filt_df[groupby_cols + agg_cols].groupby(groupby_cols).agg(['min','mean','max']).unstack(['data_layout'])
df.to_excel('results_wall_clock_times_for_data_layout.xlsx')
df